In [22]:
from datetime import date, timedelta
import gc
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb
import random
import os
import re
import json
import numpy as np
import pandas as pd
import lightgbm as lgbm
from datetime import datetime
from time import gmtime, strftime
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import torch
import lightgbm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
random_seed = 2020
random.seed(random_seed)
np.random.seed(random_seed)

In [23]:
all_data = pd.read_csv('../data/forauto.csv')
all_data = all_data.fillna('0')
all_data

# all_data[cate_cols] = all_data[cate_cols].astype("category")


,Uid,Uid_count,Category,Subcategory,Concept,Title_len,Title_number,Alltags_len,Alltags_number,img_length,...,followerCount,followingCount,Ispublic,label,img_file,Category.1,Concept.1,Subcategory.1,Alltags,Title
0,21894,4,5,65,75,33,6,128,12,333,...,2405,2729,1,11.18,train/59@N75/775.jpg,Fashion,glam,Fashion,rock punk transgender tranny electronicmusic e...,Luis Drayton - Edinburgh shoot #6
1,53866,13,0,75,139,56,13,800,65,500,...,8491,236,1,15.15,train/1@N18/1075.jpg,Travel&Active&Sports,fifa,Soccer,brazil rio brasil riodejaneiro by maria fifa m...,Arena da Barra - Arena HSBC - Arena do PAN #...
2,26948,1,3,42,480,12,2,188,23,500,...,1550,1346,1,10.99,train/351@N64/4890.jpg,Entertainment,cinema,Movies,old cinema beauty marilyn photoshop movie joke...,MARILYN 2015
3,355,1,10,68,225,18,3,61,9,500,...,1233,1115,1,8.63,train/6@N59/6568.jpg,Holiday&Celebrations,old,Birthday,pictures old family scans brothers sister 1958...,Knikkertijd - 1959
4,315,31,2,43,317,8,1,146,19,500,...,5781,13,1,11.16,train/1617@N40/7079.jpg,Food,thirsty,Drinks,hot sahara animal animals desert bottles drink...,CAMELS01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,46088,2,0,35,301,6,1,78,13,500,...,489,612,1,0.00,test/49686@N77/1118324.jpg,Travel&Active&Sports,sea,Seattle,natur nature frog sea lake fish outdoor macro ...,Quaak!
486190,12280,7,0,60,110,25,4,164,23,500,...,0,0,1,0.00,test/31147@N75/1118329.jpg,Travel&Active&Sports,cars,Cars,11 arisk okruh veternov 2016 10 jn preov veter...,arisk okruh veternov 2016
486191,43496,9,4,28,186,41,7,85,9,500,...,702,113,1,0.00,test/5656@N26/1118336.jpg,Animal,cat,Cats,holland netherlands amsterdam cat nl ramses gu...,Bike & Cat / Eerste Anjeliers Dwarsstraat
486192,5492,17,7,27,54,15,3,305,35,500,...,1092,389,1,0.00,test/28302@N36/1118349.jpg,Whether&Season,white,Snowing,street people bw white black netherlands monoc...,The Stairs crew


In [24]:
all_data.columns

Index(['Uid', 'Uid_count', 'Category', 'Subcategory', 'Concept', 'Title_len',
       'Title_number', 'Alltags_len', 'Alltags_number', 'img_length',
       'img_width', 'pixel', 'img_model', 'svd_mode_t_0', 'svd_mode_t_1',
       'svd_mode_t_2', 'svd_mode_t_3', 'svd_mode_t_4', 'svd_mode_t_5',
       'svd_mode_t_6', 'svd_mode_t_7', 'svd_mode_t_8', 'svd_mode_t_9',
       'svd_mode_t_10', 'svd_mode_t_11', 'svd_mode_t_12', 'svd_mode_t_13',
       'svd_mode_t_14', 'svd_mode_t_15', 'svd_mode_t_16', 'svd_mode_t_17',
       'svd_mode_t_18', 'svd_mode_t_19', 'svd_mode_0', 'svd_mode_1',
       'svd_mode_2', 'svd_mode_3', 'svd_mode_4', 'svd_mode_5', 'svd_mode_6',
       'svd_mode_7', 'svd_mode_8', 'svd_mode_9', 'svd_mode_10', 'svd_mode_11',
       'svd_mode_12', 'svd_mode_13', 'svd_mode_14', 'svd_mode_15',
       'svd_mode_16', 'svd_mode_17', 'svd_mode_18', 'svd_mode_19', 'Mediatype',
       'hour', 'day', 'weekday', 'week_hour', 'year_weekday', 'Longitude',
       'Latitude', 'Geoaccuracy', 'phot

In [25]:
useless_cols = ['Category.1', 'Concept.1', 'Subcategory.1', 'Alltags','Title','img_file']
all_data = all_data.drop(useless_cols, axis=1)
# train_all_data = all_data[:-180581]
# submit_all_data = all_data[-180581:]
cate_cols = ['Uid', 'Category', 'Subcategory', 'Concept', 'Mediatype', 'hour', 'day', 'weekday', 'week_hour', 'year_weekday','Geoaccuracy', 'ispro' , 'Ispublic']
all_data

,Uid,Uid_count,Category,Subcategory,Concept,Title_len,Title_number,Alltags_len,Alltags_number,img_length,...,totalFaves,totalInGroup,photoCount,meanView,meanTags,meanFaves,followerCount,followingCount,Ispublic,label
0,21894,4,5,65,75,33,6,128,12,333,...,904,176,1446,4299.470263,1.531812,0.625173,2405,2729,1,11.18
1,53866,13,0,75,139,56,13,800,65,500,...,6200,679,75883,735.884072,0.052594,0.081705,8491,236,1,15.15
2,26948,1,3,42,480,12,2,188,23,500,...,2583,2,1039,5124.185756,2.820019,2.486044,1550,1346,1,10.99
3,355,1,10,68,225,18,3,61,9,500,...,76088,385,124017,148.035342,0.046018,0.613529,1233,1115,1,8.63
4,315,31,2,43,317,8,1,146,19,500,...,0,102,9218,5300.882729,0.579735,0.000000,5781,13,1,11.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,46088,2,0,35,301,6,1,78,13,500,...,4341,84,347,833.933718,2.844380,12.510086,489,612,1,0.00
486190,12280,7,0,60,110,25,4,164,23,500,...,0,0,0,0.000000,0.000000,0.000000,0,0,1,0.00
486191,43496,9,4,28,186,41,7,85,9,500,...,5225,98,6556,1611.040268,3.334503,0.796980,702,113,1,0.00
486192,5492,17,7,27,54,15,3,305,35,500,...,24679,373,2138,2480.537418,2.655285,11.543031,1092,389,1,0.00


In [8]:
# vilt_fea = pd.read_csv('../data/vilt_fea_vqa.csv')
# all_data = pd.concat([all_data, vilt_fea], axis=1)
# all_data

,Uid,Uid_count,Category,Subcategory,Concept,Title_len,Title_number,Alltags_len,Alltags_number,img_length,...,viltvqa_fea_758,viltvqa_fea_759,viltvqa_fea_760,viltvqa_fea_761,viltvqa_fea_762,viltvqa_fea_763,viltvqa_fea_764,viltvqa_fea_765,viltvqa_fea_766,viltvqa_fea_767
0,21894,4,5,65,75,33,6,128,12,333,...,0.315131,0.239216,-0.493226,0.382592,-0.298273,0.482726,-0.329398,0.172889,0.331349,-0.215726
1,53866,13,0,75,139,56,13,800,65,500,...,-0.787129,0.377099,-0.135965,0.218942,0.218627,0.132974,0.399288,-0.266744,0.438168,0.352710
2,26948,1,3,42,480,12,2,188,23,500,...,0.174423,0.405909,0.241776,0.191860,-0.393805,0.406829,-0.494684,0.053800,0.301847,-0.274030
3,355,1,10,68,225,18,3,61,9,500,...,0.133044,-0.114484,0.387913,0.248558,-0.066610,0.157745,-0.294297,0.001803,0.410927,0.153709
4,315,31,2,43,317,8,1,146,19,500,...,0.236242,0.466902,-0.222561,-0.061981,-0.021660,0.001125,-0.149710,0.290287,-0.238527,-0.003913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,46088,2,0,35,301,6,1,78,13,500,...,-0.034823,0.254697,-0.346216,-0.202336,-0.147684,0.071354,-0.135466,-0.194680,0.166100,-0.407186
486190,12280,7,0,60,110,25,4,164,23,500,...,-0.017615,0.306144,-0.109546,-0.002779,0.084461,0.105725,0.233413,0.267870,0.317124,0.061053
486191,43496,9,4,28,186,41,7,85,9,500,...,-0.101830,-0.091368,-0.132160,0.135501,-0.167699,-0.191692,0.251878,0.057899,0.699864,0.174545
486192,5492,17,7,27,54,15,3,305,35,500,...,0.146164,-0.001863,0.322281,0.288081,-0.352652,0.434992,0.100914,0.413298,-0.327786,-0.225809


In [26]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
from scipy import stats
from scipy.stats import norm, skew
columns = ['Title_len', 'Title_number', 'Alltags_len', 'Alltags_number', 'photo_count', 'totalTags', 'totalGeotagged', 'totalFaves',
          'totalInGroup','photoCount','meanView', 'meanTags', 'meanFaves', 'followerCount','followingCount']
skew_features = all_data[columns].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skew_features[abs(skew_features) > 0.75]
skew_index = high_skew.index
for i in skew_index:
    all_data[i] = np.log1p(all_data[i])

# le = LabelEncoder()
# # all_data['family'] = le.fit_transform(all_data['family'].values)
# for cal in cate_cols:
#     all_data[cal] = le.fit_transform(all_data[cal].values)
    


In [27]:
all_data

,Uid,Uid_count,Category,Subcategory,Concept,Title_len,Title_number,Alltags_len,Alltags_number,img_length,...,totalFaves,totalInGroup,photoCount,meanView,meanTags,meanFaves,followerCount,followingCount,Ispublic,label
0,21894,4,5,65,75,3.526361,1.945910,4.859812,2.564949,333,...,6.807935,5.176150,7.277248,8.366480,0.928935,0.485614,7.785721,7.912057,1,11.18
1,53866,13,0,75,139,4.043051,2.639057,6.685861,4.189655,500,...,8.732466,6.522093,11.236961,6.602431,0.051258,0.078538,9.046880,5.468060,1,15.15
2,26948,1,3,42,480,2.564949,1.098612,5.241747,3.178054,500,...,7.857094,1.098612,6.946976,8.541922,1.340255,1.248768,7.346655,7.205635,1,10.99
3,355,1,10,68,225,2.944439,1.386294,4.127134,2.302585,500,...,11.239659,5.955837,11.728182,5.004183,0.044990,0.478424,7.118016,7.017506,1,8.63
4,315,31,2,43,317,2.197225,0.693147,4.990433,2.995732,500,...,0.000000,4.634729,9.129022,8.575817,0.457257,0.000000,8.662505,2.639057,1,11.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,46088,2,0,35,301,1.945910,0.693147,4.369448,2.639057,500,...,8.376090,4.442651,5.852202,6.727352,1.346612,2.603437,6.194405,6.418365,1,0.00
486190,12280,7,0,60,110,3.258097,1.609438,5.105945,3.178054,500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0.00
486191,43496,9,4,28,186,3.737670,2.079442,4.454347,2.302585,500,...,8.561401,4.595120,8.788288,7.385256,1.466607,0.586107,6.555357,4.736198,1,0.00
486192,5492,17,7,27,54,2.772589,1.386294,5.723585,3.583519,500,...,10.113748,5.924256,7.668094,7.816634,1.296174,2.529165,6.996681,5.966147,1,0.00


In [28]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")
import gc

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

from sklearn.decomposition import PCA
# from umap import UMAP
from sklearn.cluster import KMeans
from haversine import haversine

import optuna
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

bold = ['\033[1m', '\033[0m']



In [30]:
import gc
# del train
# del test
train_all_data = all_data[:-180581]
submit_all_data = all_data[-180581:]
train_label_df = train_all_data[['label']]
train_feature_df = train_all_data.drop(['label'], axis=1)

submit_label_df = submit_all_data[['label']]
submit_feature_df = submit_all_data.drop(['label'], axis=1)

print(len(train_feature_df), len(submit_feature_df), len(train_feature_df.columns))
print(len(train_label_df), len(submit_label_df), len(train_feature_df.columns))

305613 180581 82
305613 180581 82


In [19]:
# train_label_df

In [31]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def f_importance_plot(f_imp):
    fig = plt.figure(figsize = (15, 0.35*len(f_imp)))
    plt.title('Feature importances', size=25, y=1.05, 
              fontname='Calibri', fontweight='bold', color='#444444')
    a = sns.barplot(data=f_imp, x='avg_imp', y='feature', 
                    palette='Blues_d', linestyle="-", 
                    linewidth=1, edgecolor="black")
    plt.xlabel('')
    plt.xticks([])
    plt.ylabel('')
    plt.yticks(size=11, color='#444444')
    
    for j in ['right', 'top', 'bottom']:
        a.spines[j].set_visible(False)
    for j in ['left']:
        a.spines[j].set_linewidth(0.5)
    plt.show()

In [ ]:
import xgboost as xgb
import lightgbm as lgb
from scipy import stats
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
lgb_params = {
    'max_depth': 9,
    'learning_rate': 0.01,
    # 'min_data_in_leaf': 36, 
    # 'num_leaves': 100, 
    # 'feature_fraction': 0.8, 
    # 'bagging_fraction': 0.89, 
    # 'bagging_freq': 5, 
    # 'lambda_l2': 28,
    # 'seed': seed,
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'device': 'cpu', 
    # 'gpu_platform_id': 0,
    # 'gpu_device_id': 0,
    'n_jobs': -1,
    'metric': 'rmse',
    'verbose': -1
}

f_imp = pd.DataFrame({'feature': train_feature_df.columns})
predictions, scores = np.zeros(len(submit_feature_df)), []

k = KFold(n_splits=5, random_state=2020, shuffle=True)
for fold, (train_idx, val_idx) in enumerate(k.split(train_feature_df, train_label_df)):
    print(f'\n--- FOLD {fold+1} ---')
        
    lgb_train = lgb.Dataset(data=train_feature_df.iloc[train_idx], 
                            label=train_label_df.iloc[train_idx],
                            categorical_feature=cate_cols)
    lgb_valid = lgb.Dataset(data=train_feature_df.iloc[val_idx], 
                            label=train_label_df.iloc[val_idx],
                            categorical_feature=cate_cols,
                            reference=lgb_train)

    model = lgb.train(params=lgb_params, 
                      train_set=lgb_train, 
                      num_boost_round=50000,
                      valid_sets=[lgb_train, lgb_valid], 
                      valid_names=['train', 'val'],
                      verbose_eval=1000,
                      callbacks=[lgb.log_evaluation(1000),
                                 lgb.early_stopping(1000, verbose=False)])
    
    f_imp['fold_'+str(fold+1)] = model.feature_importance()
    b_itr = model.best_iteration
    
    val_preds = model.predict(train_feature_df.iloc[val_idx], num_iteration=b_itr)
    val_score = rmse(train_label_df['label'].iloc[val_idx], val_preds)
    val_mae = mean_squared_error(train_label_df['label'].iloc[val_idx], val_preds)
    val_src = stats.spearmanr(train_label_df['label'].iloc[val_idx], val_preds)[0]
    scores.append(val_score)
    print("valid: MSE: %.4f, MAE: %.4f, SRC: %.4f"%(val_score, val_mae,val_src))
    predictions += model.predict(submit_feature_df, num_iteration=b_itr) / 5
    print(f'--- RMSE: {bold[0]}{round(val_score, 6)}{bold[1]} | best iteration: {bold[0]}{b_itr}{bold[1]} ---')
    
    del lgb_train, lgb_valid, val_preds, val_score, model
    gc.collect()

print('*'*45)
print(f'Mean RMSE: {bold[0]}{round(np.mean(scores), 6)}{bold[1]}')

f_imp['avg_imp'] = f_imp[f_imp.columns[1:]].mean(axis=1)
f_imp.sort_values('avg_imp', ascending=False, inplace=True)
f_importance_plot(f_imp)


--- FOLD 1 ---


In [ ]:
lgbm_preds = predictions.copy()
ss['LGBM_pred'] = predictions
ss.to_csv('lgbm_pred.csv', index=False)